# Search Engine Tools  

 > A library of classes that provide query access to a number of online academic search services including (Pubmed, PMC, and European PMC). 

In [ ]:
#| default_exp utils.searchEngineUtils

In [ ]:
#| hide
from nbdev import *

## NCBI Tools

In [ ]:
#|export

import alhazen.schema_python as linkml_py
from alhazen.schema_sqla import *
from alhazen.utils.web_robot import retrieve_full_text_links_from_biorxiv, \
                retrieve_pdf_from_doidotorg, \
                get_html_from_pmc_doi
from bs4 import BeautifulSoup,Tag,Comment,NavigableString
import datetime
from enum import Enum
import json
import os
import pandas as pd
from pathlib import Path
from pyalex import Works, Authors, Sources, Institutions, Concepts, Publishers, Funders, config
import re
import requests
from time import time,sleep
from tqdm import tqdm
from urllib.request import urlopen
from urllib.parse import quote_plus, quote, unquote
from urllib.error import URLError
import uuid

In [ ]:
#|export

PAGE_SIZE = 10000
TIME_THRESHOLD = 0.3333334

class NCBI_Database_Type(Enum):
  """
  Simple enumeration of the different NCBI databases supported by this tool
  """
  pubmed = 'pubmed'
  PMC = 'PMC'

class ESearchQuery:
  """
  Class to provide query interface for ESearch (i.e., query terms in elaborate ways, return a list of ids)
  Each instance of this class executes queries of a given type
  """

  def __init__(self, api_key=None, oa=False, db='pubmed'):
    """
    :param api_key: API Key for NCBI EUtil Services 
    :param oa: Is this query searching for open access papers? 
    :param db: The database being queried
    """
    self.api_key = api_key
    self.idPrefix = ''
    self.oa = oa
    self.db = db
  
  def build_query_tuples(self, df, check_threshold, name_col, terms_col, sep):
    '''
    Given a dataframe defining a set of 'OR' queries (i.e., entirely broken into | clauses), 
    check each indvidual term in Pubmed and return complete queries with problematic terms 
    stripped.
    '''
    query_tuples = []
    phrase_counts = []
    for ind in df.index:
      search_l = []
      terms_to_check = [df[name_col][ind]]
      terms_to_check.extend(df[terms_col][ind].split('|'))
      for s in terms_to_check: 
        go_no_go, phrase, count = _check_query_phrase(self, s.strip())
        print(go_no_go, phrase, count)
        if go_no_go:
          search_l.append(phrase)
          sleep(0.10)
          if count>check_threshold:
            phrase_counts.append((phrase, count))
      query_tuples.append( (ind, df[name_col][ind], ' OR '.join(search_l)) )
    return query_tuples, phrase_counts
  
  def _check_query_phrase(self, phrase):
    """
    Checks whether a phrase would work on Pubmed or would be expanded (which can lead to unpredictable errors). 
    Use this as a check for synonyms.   
    """
    idPrefix = ''
    m1 = re.match('^[a-zA-Z0-9]{1,5}$', phrase)
    if m1 is not None:
      return False, 'Abbreviation', 0

    m2 = re.search('[(\)]', phrase)
    if m2 is not None:
      return False, 'Brackets', 0

    m3 = re.search('[\,\;]', phrase)
    if m3 is not None:
      phrase = '("' + '" AND "'.join(re.split('[\,\;]', phrase.strip()))+'")'

    if self.api_key is not None: 
      esearch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key='+self.api_key+'&db=' + self.db + '&term='
    else:
      esearch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db='+self.db + '&term='
    url =  esearch_stem + quote('"'+phrase+'"')

    esearch_response = urlopen(url)
    esearch_data = esearch_response.read().decode('utf-8')
    esearch_soup = BeautifulSoup(esearch_data, "lxml-xml")
    count = int(esearch_soup.find('Count').string)
    #n_translations = len(esearch_soup.find('TranslationStack').findAll('TermSet'))
    phrase_not_found = esearch_soup.find('PhraseNotFound')
    quoted_phrase_not_found = esearch_soup.find('QuotedPhraseNotFound')
    if phrase_not_found is not None or quoted_phrase_not_found is not None:
      return False, '"'+phrase+'" not found', 0
    if count == 0:
      return False, phrase, count
    return True, phrase, count        

  def execute_count_query(self, query):
    """
    Executes a query on the target database and returns a count of papers 
    """
    idPrefix = ''
    if self.oa:
      if self.db == NCBI_Database_Type.PMC:
        query = '"open access"[filter] AND (' + query + ')'
        idPrefix = 'PMC'
      elif self.db == NCBI_Database_Type.pubmed:
        query = '"loattrfree full text"[sb] AND (' + query + ')'
    if self.api_key: 
      esearch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key='+self.api_key+'&db=' + self.db + '&term='
    else:
      esearch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db='+self.db + '&term='
    query = quote_plus(query)
    esearch_response = urlopen(esearch_stem + query)
    esearch_data = esearch_response.read().decode('utf-8')
    esearch_soup = BeautifulSoup(esearch_data, "lxml-xml")
    count_tag = esearch_soup.find('Count')
    if count_tag is None:
      raise Exception('No Data returned from "' + self.query + '"')
    return int(count_tag.string)

  def execute_query_on_website(self, q, pm_order='relevance'):
    """
    Executes a query on the Pubmed database and returns papers in order of relevance or date.
    This is important to determine accuracy of complex queries are based on the composition of the first page of results.  
    """
    query = 'https://pubmed.ncbi.nlm.nih.gov/?format=pmid&size=10&term='+re.sub('\s+','+',q)
    if pm_order == 'date':
      query += '&sort=date'
    response = urlopen(query)
    data = response.read().decode('utf-8')
    soup = BeautifulSoup(data, "lxml-xml")
    pmids = re.split('\s+', soup.find('body').text.strip())
    return pmids

  def execute_query(self, query):
    """
    Executes a query on the eutils service and returns data as a Pandas Dataframe
    """
    idPrefix = ''
    if self.oa:
      if self.db == NCBI_Database_Type.PMC:
        query = '"open access"[filter] AND (' + query + ')'
        idPrefix = 'PMC'
      elif self.db == NCBI_Database_Type.pubmed:
        query = '"loattrfree full text"[sb] AND (' + query + ')'

    if self.api_key: 
      esearch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key='+self.api_key+'&db=' + self.db + '&term='
    else: 
      esearch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=' + self.db + '&term='

    query = quote_plus(query)
    print(esearch_stem + query)
    esearch_response = urlopen(esearch_stem + query)
    esearch_data = esearch_response.read().decode('utf-8')
    esearch_soup = BeautifulSoup(esearch_data, "lxml-xml")
    count_tag = esearch_soup.find('Count')
    if count_tag is None:
        raise Exception('No Data returned from "' + query + '"')
    count = int(count_tag.string)

    latest_time = time()

    ids = []
    for i in tqdm(range(0,count,PAGE_SIZE)):
      full_query = esearch_stem + query + '&retstart=' + str(i)+ '&retmax=' + str(PAGE_SIZE)
      esearch_response = urlopen(full_query)
      esearch_data = esearch_response.read().decode('utf-8')
      esearch_soup = BeautifulSoup(esearch_data, "lxml-xml")
      for pmid_tag in esearch_soup.find_all('Id') :
        ids.append(self.idPrefix + pmid_tag.text)
      delta_time = time() - latest_time
      if delta_time < TIME_THRESHOLD :
        sleep(TIME_THRESHOLD - delta_time)
        
    return ids

In [ ]:
#|export
class EFetchQuery:
    """
    Class to provide query interface for EFetch (i.e., query based on a list of ids)
    Each instance of this class executes queries of a given type
    """

    def __init__(self, api_key=None, db='pubmed'):
        """
        :param api_key: API Key for NCBI EUtil Services 
        :param oa: Is this query searching for open access papers? 
        :param db: The database being queried
        """
        self.api_key = api_key
        self.db = db

    def execute_efetch(self, id):
        """
        Executes a query for a single specific identifier
        """
        if self.api_key:
          efetch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?api_key='+self.api_key+'&db=pubmed&retmode=xml&id='
        else:
          efetch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id='
        efetch_response = urlopen(efetch_stem + str(id))
        return self._generate_rows_from_medline_records(efetch_response.read().decode('utf-8'))

    def generate_data_frame_from_id_list(self, id_list):
        """
        Executes a query for a list of ID values
        """
        if self.api_key:
          efetch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?api_key='+self.api_key+'&db=pubmed&retmode=xml&id='
        else:
          efetch_stem = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id='
        page_size = 100
        i = 0
        url = efetch_stem
        efetch_df = pd.DataFrame()
        for line in tqdm(id_list):
            try:
                l = re.split('\s+', str(line))
                pmid = l[0]
                i += 1
                if i >= page_size:
                    efetch_response = urlopen(url)
                    df = self._generate_rows_from_medline_records(efetch_response.read().decode('utf-8'))
                    efetch_df = efetch_df.append(df)
                    url = efetch_stem
                    i = 0

                if re.search('\d$', url):
                    url += ','
                url += pmid.strip()
            except URLError as e:
                sleep(10)
                print("URLError({0}): {1}".format(e.errno, e.strerror))
            except TypeError as e2:
                pause = 1

        if url != efetch_stem:
            efetch_response = urlopen(url)
            df = self._generate_rows_from_medline_records(efetch_response.read().decode('utf-8'))
            efetch_df = efetch_df.append(df)
        return efetch_df

    def generate_mesh_data_frame_from_id_list(self, id_list):
        """
        Executes a query for MeSH data from a list of ID values
        """
        url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi'
        if self.api_key:
          payload = 'api_key='+self.api_key+'&db=pubmed&retmode=xml&id='
        else:
          payload = 'db=pubmed&retmode=xml&id='
          
        headers = {'content-type': 'application/xml'}

        page_size = 10000
        i = 0
        efetch_df = pd.DataFrame()
        for line in tqdm(id_list):
            try:
                l = re.split('\s+', line)
                pmid = l[0]
                i += 1
                if i >= page_size:
                    print('   running query')
                    r = requests.post(url, data=payload)
                    efetch_data = r.content.decode('utf-8')
                    df = self._generate_mesh_rows_from_medline_records(efetch_data)
                    efetch_df = efetch_df.append(df)
                    payload = 'db=pubmed&retmode=xml&id='
                    i = 0

                if re.search('\d$', payload):
                    payload += ','
                payload += pmid.strip()
            except URLError as e:
                sleep(10)
                print("URLError({0}): {1}".format(e.errno, e.strerror))

        if payload[-1] != '=':
            r = requests.post(url, data=payload)
            efetch_data = r.content.decode('utf-8')
            df = self._generate_mesh_rows_from_medline_records(efetch_data)
            efetch_df = efetch_df.append(df)

        return efetch_df

    def _generate_mesh_rows_from_medline_records(self, record):

        soup2 = BeautifulSoup(record, "lxml-xml")

        rows = []
        cols = ['PMID','MESH']
        for citation_tag in tqdm(soup2.find_all('MedlineCitation')):

            pmid_tag = citation_tag.find('PMID')

            mesh_labels = []

            if pmid_tag is None:
                continue

            for meshTag in citation_tag.findAll('MeshHeading'):
                desc = meshTag.find('DescriptorName')
                qual_list = meshTag.findAll('QualifierName')
                if len(qual_list)>0:
                    mesh_labels.append('%s/%s'%(desc.text.replace('\n', ' '),'/'.join(q.text.replace('\n', ' ') for q in qual_list)))
                else:
                    mesh_labels.append(desc.text.replace('\n', ' '))
            mesh_data = ",".join(mesh_labels)

            rows.append((pmid_tag.text, mesh_data))

        df = pd.DataFrame(data=rows, columns=cols)
        return df

    def _generate_rows_from_medline_records(self, record):

        soup2 = BeautifulSoup(record, "lxml-xml")

        rows = []
        cols = ['PMID', 'YEAR', 'PUBLICATION_TYPE', 'TITLE', 'ABSTRACT','MESH','KEYWORDS']
        for citation_tag in soup2.find_all('MedlineCitation'):

            pmid_tag = citation_tag.find('PMID')
            title_tag = citation_tag.find('ArticleTitle')
            abstract_txt = ''
            for x in citation_tag.findAll('AbstractText'):
                if x.label is not None:
                    abstract_txt += ' ' + x.label + ': '
                abstract_txt += x.text

            mesh_labels = []
            for meshTag in citation_tag.findAll('MeshHeading'):
                desc = meshTag.find('DescriptorName')
                qual_list = meshTag.findAll('QualifierName')
                if len(qual_list)>0:
                    mesh_labels.append('%s/%s'%(desc.text.replace('\n', ' '),'/'.join(q.text.replace('\n', ' ') for q in qual_list)))
                else:
                    mesh_labels.append(desc.text.replace('\n', ' '))
            mesh_data = ",".join(mesh_labels)

            if pmid_tag is None or title_tag is None or abstract_txt == '':
                continue

            year_tag = citation_tag.find('PubDate').find('Year')

            year = ''
            if year_tag is not None:
                year = year_tag.text

            is_review = '|'.join([x.text for x in citation_tag.findAll('PublicationType')])

            keyword_data = '|'.join([meshTag.text.replace('\n', ' ') for meshTag in citation_tag.findAll('Keyword')])

            rows.append((pmid_tag.text, year, is_review, title_tag.text, abstract_txt, mesh_data, keyword_data))

        df = pd.DataFrame(data=rows, columns=cols)
        return df


In [ ]:
#| export 

def download_file(url, local_filename):
    """Downloads a file from an URL to a local disk"""
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename

def get_nxml_from_pubmed_doi(doi, base_file_path):    
    """
    Executes a query on the target database and returns a count of papers 
    """
    if os.environ.get('NCBI_API_KEY') is None:
        raise Exception('Error attempting to query NCBI for URL data, did you set the NCBI_API_KEY environment variable?')
    api_key = os.environ.get('NCBI_API_KEY')

    esearch_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key='+api_key+'&db=pmc&term='+doi+'[doi]&retmode=xml'
    sleep(0.1)
    #print(esearch_url)
    esearch_response = urlopen(esearch_url)
    esearch_data = esearch_response.read().decode('utf-8')
    esearch_soup = BeautifulSoup(esearch_data, "lxml-xml")
    id_tag = esearch_soup.find('Id')    
    if id_tag is None:
      print('\t{} does not exist'.format(doi))
      return
      # raise Exception('Could not find "' + doi + '" in PMC')
    pmc_id = id_tag.string
    
    efetch_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?api_key='+api_key+'&db=pmc&id='+pmc_id+'&retmode=xml'
    sleep(0.1)
    #print(efetch_url)
    efetch_response = urlopen(efetch_url)
    efetch_data = efetch_response.read().decode('utf-8')
    xml = BeautifulSoup(efetch_data, "lxml-xml")
    body_tag = xml.findAll('body')
    if body_tag is None:
        print('\t{} no full text as NXML file.'.format(doi))
        return    
    
    file_path = Path(base_file_path + '/' + doi + '.nxml')
    parent_dir = file_path.parent
    if os.path.exists(parent_dir) is False:
        os.makedirs(parent_dir)
    with open(file_path, 'w') as f:
        f.write(str(xml))

def get_pdf_from_pubmed_doi(doi, base_file_path):    
    """
    Executes a query on the target database and returns a count of papers 
    """
    if os.environ.get('NCBI_API_KEY') is None:
        raise Exception('Error attempting to query NCBI for URL data, did you set the NCBI_API_KEY environment variable?')
    api_key = os.environ.get('NCBI_API_KEY')

    esearch_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?api_key='+api_key+'&db=pmc&term='+doi+'[doi]&retmode=xml'
    sleep(0.1)
    #print(esearch_url)
    esearch_response = urlopen(esearch_url)
    esearch_data = esearch_response.read().decode('utf-8')
    esearch_soup = BeautifulSoup(esearch_data, "lxml-xml")
    id_tag = esearch_soup.find('Id')    
    if id_tag is None:
        print('\t{} does not exist'.format(doi))
        return
      # raise Exception('Could not find "' + doi + '" in PMC')
    pmc_id = id_tag.string

    # OA Dataset 
    oapi_url = 'https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi?id='+pmc_id+'&format=pdf'
    sleep(0.1)
    print(oapi_url)
    oapi_response = urlopen(oapi_url)
    oapi_data = oapi_response.read().decode('utf-8')    
    oapi_soup = BeautifulSoup(oapi_data, "lxml-xml")
    
    pdf_link_tag = oapi_soup.find('link')
    if(pdf_link_tag is None):
        print('\t{} no full text as PDF file.'.format(doi))
        return
    
    pdf_url = pdf_link_tag['href']
    if pdf_url.startswith('ftp:'):
        pdf_url = pdf_url.replace('ftp:','https:') 
    file_path = Path(base_file_path + '/' + doi + '.pdf')
    parent_dir = file_path.parent
    if os.path.exists(parent_dir) is False:
        os.makedirs(parent_dir)
    download_file(pdf_url, file_path)   

## EuroPMCQuery

In [ ]:
#| export
class EuroPMCQuery():
    """
    A class that executes search queries on the European PMC API 
    """
    def __init__(self, oa=False, db='pubmed'):
        """
        Initialization of the class
        :param oa:
        """
        self.oa = oa

    def run_empc_query(self, q, page_size=1000, timeout=60, 
                       extra_columns=["id", "source", "doi", 'title', 'pubYear', 'abstractText', 'pubType']):
        EMPC_API_URL = 'https://www.ebi.ac.uk/europepmc/webservices/rest/search?format=JSON&pageSize='+str(page_size)+'&synonym=TRUE'
        if len(extra_columns)>0:
            EMPC_API_URL += '&resultType=core'
        url = EMPC_API_URL + '&query=' + q
        r = requests.get(url, timeout=timeout)
        data = json.loads(r.text)
        numFound = data['hitCount']
        print(url + ', ' + str(numFound) + ' European PMC PAPERS FOUND')
        publications = []
        fragments = []
        selectors = []
        cursorMark = '*'
        for i in tqdm(range(0, numFound, page_size)):
            url = EMPC_API_URL + '&cursorMark=' + cursorMark + '&query=' + q
            r = requests.get(url, timeout=timeout)
            data = json.loads(r.text)
            date_format = '%Y-%m-%d'
            #print(data)
            if data.get('nextCursorMark'):
                cursorMark = data['nextCursorMark']
            for d in data['resultList']['result']:
                pTypes = [t.lower() for t in d.get('pubTypeList',{}).get('pubType',[])]
                if d.get('firstPublicationDate'):
                    date_obj = datetime.datetime.strptime(d.get('firstPublicationDate'), date_format)
                elif d.get('dateOfCreation'):
                    date_obj = datetime.strptime(d.get('dateOfCreation'), date_format)
                else:
                    date_obj = None
                xref = ['epmid:%s'%(str(d['id']))]
                if d.get('doi'):
                    xref.append('doi:'+d.get('doi'))
                else:
                    continue # skip if no DOI
                title = d.get('title','')
                abstract = d.get('abstractText','')
                content = title+'\n'+abstract
                if 'patent' in pTypes:
                    continue
                elif 'clinical trial' in pTypes: 
                    ptype='ClinicalTrial'
                elif 'review' in pTypes or 'systematic review' in pTypes or 'systematic-review' in pTypes or 'meta-analysis' in pTypes or 'review-article' in pTypes:
                    ptype='ScientificReviewArticle'
                elif 'preprint' in pTypes:
                    ptype='ScientificPrimaryResearchPreprint'
                elif 'journal article' in pTypes or 'research-article' in pTypes:
                    ptype='ScientificPrimaryResearchArticle'
                elif 'case-report' in pTypes or 'case reports' in pTypes:
                    ptype='ClinicalCaseReport'
                elif 'practice guideline' in pTypes:
                    ptype='ClinicalGuidelines'
                elif 'letter' in pTypes or 'comment' in pTypes or 'editorial' in pTypes:
                    ptype='ScientificComment'
                elif 'published erratum' in pTypes or 'correction' in pTypes or 'retraction of publication' in pTypes: 
                    ptype='ScientificErrata'
                else:
                    #print('|'.join(pTypes))
                    continue 
                human_readable_reference = d.get('authorString','') + ' (' + str(date_obj.year) + ') ' + title
                p = ScientificKnowledgeExpression(
                        id='doi:%s'%(str(d['doi'])), 
                        iri=['doi:%s'%(str(d['doi']))], 
                        xref=xref, 
                        publication_date=date_obj, 
                        content=human_readable_reference,
                        type=ptype)
                publications.append(p)
                item = ScientificKnowledgeItem(
                        id=uuid.uuid4().hex[0:10], 
                        content=content, 
                        type='CitationRecord')
                p.has_representation.append(item)
                item.representation_of = p.id
                frg1 = ScientificKnowledgeFragment( 
                    id=item.id+'.0',
                    content=title,
                    offset=0,
                    length=len(title),
                    type='title')
                frg1.part_of = item
                frg2 = ScientificKnowledgeFragment( 
                    id=item.id+'.1',
                    content=abstract,
                    offset=len(title)+1,
                    length=len(abstract),
                    type='abstract')
                frg2.part_of = item
                item.has_part = [frg1, frg2]

        print(' Returning '+str(len(publications)))
        return (numFound, publications)

# Open Alex

In [ ]:
#| export
def read_authors_and_institutions_from_openalex(doi_list):
    oa_works = []
    for doi in doi_list:
        w = Works()[doi]        
        oa_works.append(w)
    return oa_works

# Map the OpenAlex data to the Alhazen schema
def load_paper_from_openalex(doi_or_oa_id):

    if doi_or_oa_id.startswith('https://doi.org/'):
        doi_or_oa_id = re.sub('https://doi.org/', 'doi:', doi_or_oa_id)
    elif doi_or_oa_id.startswith('doi:') is False:
        doi_or_oa_id = 'doi:'+ doi_or_oa_id
    elif doi_or_oa_id.startswith('https://openalex.org/'):
        doi_or_oa_id = re.sub('https://openalex.org/', '', doi_or_oa_id)

    try:
        w = Works()[doi_or_oa_id]
    except:
        w = None

    if w is None:
        return None

    doi = re.sub('https://doi.org/', 'doi:', w.get('doi'))
    
    authors = [] 
    orgs = set()

    content = ''
    first_author = None
    last_author = None
    middle_author = None

    for aa in w['authorships']:
        if aa.get('author_position') == 'first':
            first_author = aa
        elif aa.get('author_position') == 'last':
            last_author = aa
        elif aa.get('author_position') == 'middle':
            middle_author = aa

        a = aa.get('author', {})
        a_id = a.get('id', '')
        a_name = a.get('display_name', '')
        orcid = a.get('orcid')
        author_ice = Author(id=a_id, name=a_name, type='Author')
        authors.append(author_ice)
        if orcid:
            author_ice.xref.append(orcid)

        for inst in aa.get('institutions', []):
            inst_id = inst.get('id', '')
            inst_country = inst.get('country_code', '')
            inst_name = inst.get('display_name', '')
            ror = inst.get('ror')        
            org_ice = Organization(id=inst_id, 
                                    name=inst_name,
                                    type='Organization',
                                    country=[inst_country])
            if ror:
                org_ice.xref.append(ror)
            author_ice.affiliations.append(org_ice)
            orgs.add(org_ice)

    # First author only    
    if first_author and middle_author is None and last_author is None:
        content = str(first_author.get('raw_author_name','')) + ' (' + str(w.get('publication_year', '')) + ') ' + str(w.get('title', ''))
    # First and second author only    
    elif first_author and middle_author is None and last_author:
        content = str(first_author.get('raw_author_name',''))+' and '+str(last_author.get('raw_author_name', ''))+' ('+str(w.get('publication_year', ''))+') '+str(w.get('title', ''))
    # 'et al.'    
    elif first_author:
        content = str(first_author.get('raw_author_name', ''))+' et al. ('+str(w.get('publication_year', ''))+') '+str(w.get('title', ''))
    else:
        return None

    xrefs = set()
    oa = w.get('ids',{}).get('openalex')
    if oa:
        xrefs.add(oa)
    doi2 = w.get('ids',{}).get('doi')
    if doi2:
        xrefs.add(doi2)
    pmid = w.get('ids',{}).get('pmid')
    if pmid:
        xrefs.add(pmid)        
    pmcid = w.get('ids',{}).get('pmcid')
    if pmcid:
        xrefs.add(pmcid)

    title = w.get('title')
    abstract = w['abstract']
    if abstract is None:
        abstract = ''  

    # How to match the type of the paper to the Alhazen schema? 
    e = ScientificKnowledgeExpression(id=doi,    
                                 xref=list(xrefs), 
                                 publication_date=w.get('publication_date'), 
                                 content=content,
                                 type='ScientificPrimaryResearchArticle')
    
    #for a in authors:
    #    e.has_authors.append(a)
    #    a.is_author_of.append(e)

    item = ScientificKnowledgeItem(
            id=uuid.uuid4().hex[0:10], 
            content=content, 
            type='CitationRecord')
    e.has_representation.append(item)
    item.representation_of = e.id
    frg1 = ScientificKnowledgeFragment( 
        id=item.id+'.0',
        content=title,
        offset=0,
        length=len(title),
        type='title')
    frg1.part_of = item
    frg2 = ScientificKnowledgeFragment( 
        id=item.id+'.1',
        content=abstract,
        offset=len(title)+1,
        length=len(abstract),
        type='abstract')
    frg2.part_of = item
    item.has_part = [frg1, frg2]

    return e

# Read a papers references fromv OpenAlex
def read_references_from_openalex(doi_or_oa_id):

    if doi_or_oa_id.startswith('https://doi.org/'):
        doi_or_oa_id = re.sub('https://doi.org/', 'doi:', doi_or_oa_id)
    elif doi_or_oa_id.startswith('doi:') is False:
        doi_or_oa_id = 'doi:'+ doi_or_oa_id
    elif doi_or_oa_id.startswith('https://openalex.org/'):
        doi_or_oa_id = re.sub('https://openalex.org/', '', doi_or_oa_id)

    w = Works()[doi_or_oa_id]

    if w is None:
        return None
    
    referenced_works = w.get('referenced_works')

    return referenced_works


# Full text downloads from multiple sources

In [ ]:
#| export
def download_full_text_paper_for_doi(doi, path, headless=True): 
    """Attempts to download full text for a given DOI if the paper is listed as open access."""

    if path[-1:] != '/':
        path += '/'
    nxml_file_path = path+doi+'.nxml'
    pdf_file_path = path+doi+'.pdf'
    html_file_path = path+doi+'.html'
    
    if os.path.exists(nxml_file_path) or os.path.exists(pdf_file_path) or os.path.exists(html_file_path):
        return True
    
    # Check if the paper is open access from UnPayWall
    dev_email = os.environ.get('DEV_EMAIL', 'gully.burns@chanzuckerberg.com') 
    stem_url = 'https://api.unpaywall.org/v2/'
    resp = requests.get(stem_url+doi+'?email='+dev_email).json()
    if resp.get('is_oa', False) is False:
        return False

    try:
        # Try NCBI open access NXML first
        ft_path = get_nxml_from_pubmed_doi(doi, path)
        if ft_path is not None:
            return True

        # If the DOI starts with, '10.1101/', it's a bioRxiv paper, so go get it there. 
        if doi.startswith('10.1101/'):
            ft_path = retrieve_full_text_links_from_biorxiv(doi, path)
            if ft_path is not None:
                return True
            
        # Try NCBI open access PDF
        ft_path = get_pdf_from_pubmed_doi(doi, path)
        if ft_path is not None:
            return True

        # Try to get the pdf from the doi
        ft_path = retrieve_pdf_from_doidotorg(doi, path, headless=headless)
        if ft_path is not None:
            return True
        
        # Finally, try to find an NCBI HTML page for the paper 
        ft_path = get_html_from_pmc_doi(doi, path)
        if ft_path is not None:
            return True
        
    except Exception as e:
        print(e)
        
    return False